In [1]:
#################################################
#created the 17/05/2018 14:14 by Alexis Blanchet#
#################################################
#-*- coding: utf-8 -*-
'''

'''

'''
Améliorations possibles:

'''
############ display options ####################
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

import pandas as pd
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
#################################################
###########        Imports      #################
#################################################
import pandas as pd
import numpy as np

In [2]:
date = "2018-05-23"

PTV = pd.read_csv('IPTV_'+date+'_TF1.csv')[['TITRE','DUREE','description programme','HEURE',
       'debut']]
PTV['fin'] = PTV['debut']+PTV['DUREE']
Points = pd.read_csv('merged_'+date+'.csv')[['minutes']]
true_labels = pd.read_csv('label_'+date+'.csv',header=None)


PTV

Points

In [3]:
begin = (pd.DataFrame(PTV.loc[(PTV.shape[0]-1)]).T).reset_index().drop(['index'],axis=1)

begin

In [13]:
verbose = False
label = []
#Initialisation des Variables
index_CP = 0
index_PTV = PTV.shape[0]-1
##############
lastCP = 200
lastPub= 0
lastend = 130
lastduree = 255
planifiedend = 385
begin = True
#Initialisation du NewPTV
OnlinePTV = pd.DataFrame()
OnlinePTV['minute'] = [180]
OnlinePTV['TITRE'] = 'Programmes de la nuit'
OnlinePTV['Change Point'] = 'Non'
OnlinePTV['pourcentage de la durée'] = 0
OnlinePTV['Évenement'] = 'Début de Détection'
OnlinePTV


,minute,TITRE,Change Point,pourcentage de la durée,Évenement
0,180,Programmes de la nuit,Non,0,Début de Détection


In [14]:


for i in range(180,1620):
    
    #incrémentation
    lastPub+=1
    lastCP+=1
    if(index_CP==Points.shape[0]):
        index_CP -=1
    if(i==Points['minutes'][index_CP]):
        #print(index_CP,len(label),Points.shape[0])
        #print(i,Points["minutes"].iloc[index_CP])
        index_CP+=1
        if(lastCP<4):
            if(verbose):
                print("Détection multiple d'un même événement")
            label.append(label[-1])
        else:
            percentage = (i%1440-lastend)/lastduree
            if(i%1440-lastend<0):
                percentage = (i-lastend)/lastduree
            if(planifiedend>i):
                #On détermine la10 place du CP dans le programme
                if(lastduree<10):
                    if(verbose):
                        print("Change Point en fin de programme")
                    if((i-lastend)<10):
                        OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,PTV['TITRE'].iloc[index_PTV],'oui',percentage,"fin d'un programme court"]
                        lastend = i+5
                        lastCP=0
                        index_PTV += 1
                        index_PTV = index_PTV%(PTV.shape[0])
                        lastduree = PTV['DUREE'].iloc[index_PTV]
                        planifiedend = (lastend + lastduree)
                        label.append(2)
                else:
                    if(percentage<=0.25):
                        if(verbose):
                            print("Change Point en début de programme")
                        if(lastPub>20):
                            OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,"publicité",'oui',percentage,"publicité dans la première partie d'un programme"]
                            label.append(1)
                            lastCP=0
                            lastPub = 0
                        else:
                            if(verbose):
                                print("Faux Change Point en début de programme")
                            label.append(0)


                    elif(percentage>0.75 and PTV['TITRE'].iloc[index_PTV] != 'Journal'):
                        if(verbose):
                            print("Change Point en fin de programme")
                        if((i-lastend)<15):
                            OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,PTV['TITRE'].iloc[index_PTV],'oui',percentage,"fin d'un programme court"]
                            lastend = i+5
                            lastCP=0
                            index_PTV += 1
                            index_PTV = index_PTV%(PTV.shape[0])
                            lastduree = PTV['DUREE'].iloc[index_PTV]
                            planifiedend = (lastend + lastduree)
                            label.append(2)
                        elif(planifiedend-i<10):
                            OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,PTV['TITRE'].iloc[index_PTV],'oui',percentage,"fin d'un programme long"]
                            lastend = i
                            lastCP=0
                            index_PTV += 1  
                            index_PTV = index_PTV%(PTV.shape[0])
                            lastduree = PTV['DUREE'].iloc[index_PTV]
                            planifiedend = (lastend + lastduree)
                            label.append(2)
                        elif(lastPub>20):
                            OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,"publicité",'oui',percentage,"publicité dans la première partie d'un programme"]
                            lastCP=0
                            lastPub = 0
                            label.append(1)
                        else:
                            if(verbose):
                                print("Faux Change Point en Début de Programme")
                            label.append(0)
                    elif(percentage>0.9):
                        if(verbose):
                            print("Change Point en fin de programme")
                        if((i-lastend)<10):
                            OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,PTV['TITRE'].iloc[index_PTV],'oui',percentage,"fin d'un programme court"]
                            lastend = planifiedend
                            lastCP=0
                            index_PTV += 1
                            index_PTV = index_PTV%(PTV.shape[0])
                            lastduree = PTV['DUREE'].iloc[index_PTV]
                            planifiedend = (lastend + lastduree)
                            label.append(2)
                        elif(planifiedend-i<6):
                            OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,PTV['TITRE'].iloc[index_PTV],'oui',percentage,"fin d'un programme long"]
                            lastend = planifiedend
                            lastCP=0
                            index_PTV += 1  
                            index_PTV = index_PTV%31
                            lastduree = PTV['DUREE'].iloc[index_PTV]
                            planifiedend = (lastend + lastduree)
                            label.append(2)
                        elif(lastPub>20):
                            OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,"publicité",'oui',percentage,"publicité dans la première partie d'un programme"]
                            lastCP=0
                            lastPub = 0
                            label.append(1)
                        else:
                            if(verbose):
                                print("Faux Change Point en Début de Programme")
                            label.append(0)
                        
                
                    else:
                        if(verbose):
                            print("Change Point en milieu de programme")
                        if(lastPub>20):
                            OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,"publicité",'oui',percentage,"publicité au milieu d'un programme"]
                            lastCP=0
                            lastPub = 0
                            label.append(1)
                        else:
                            if(verbose):
                                print("Faux Change Point en début de programme")
                            label.append(0)

                    
                
            
            
            else:
                if(verbose):
                    print("Change Point à l'éxtérieur du programme en cours")
                if(i>lastend+lastduree+(min(20,max(0.2*lastduree,5)))):
                    OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,PTV['TITRE'].iloc[index_PTV],'oui',percentage,"fin non détectée d'un programme"]
                    lastend = i
                    lastCP=0
                    index_PTV += 1  
                    index_PTV = index_PTV%(PTV.shape[0])
                    lastduree = PTV['DUREE'].iloc[index_PTV]
                    planifiedend = (lastend + lastduree)
                    label.append(2)
                elif(i<=lastend+int(1.2*lastduree)+1):
                    if(lastduree<15):
                        OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,PTV['TITRE'].iloc[index_PTV],'oui',percentage,"fin d'un programme court"]
                        lastend = i
                        lastCP=0
                        index_PTV += 1  
                        index_PTV = index_PTV%(PTV.shape[0])
                        lastduree = PTV['DUREE'].iloc[index_PTV]
                        planifiedend = (lastend + lastduree)
                        label.append(2)
                    else:   
                        OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,PTV['TITRE'].iloc[index_PTV],'oui',percentage,"fin d'un programme long"]
                        lastend = i+6
                        lastCP=0
                        index_PTV += 1  
                        index_PTV = index_PTV%(PTV.shape[0])
                        lastduree = PTV['DUREE'].iloc[index_PTV]-3
                        planifiedend = (lastend + lastduree)
                        label.append(2)
                    
                
            
        
        
        
            
    else:
        percentage = (i%1440-lastend)/lastduree
        if(i%1440-lastend<0):
            percentage = (i-lastend)/lastduree
        if(lastduree<15 and i>lastend+lastduree):
            if(verbose):
                print("fin non détectée d'un programme (rattrapage hors CP)")
            OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,PTV['TITRE'].iloc[index_PTV],'non',percentage,"fin non détectée d'un programme court(rattrapage hors CP)"]
            lastend = planifiedend
            lastCP=0
            index_PTV += 1  
            index_PTV = index_PTV%(PTV.shape[0])
            lastduree = PTV['DUREE'].iloc[index_PTV]
            planifiedend = (lastend + lastduree)
            
        if(i>lastend+lastduree+(min(20,max(0.2*lastduree,5)))):
            if(verbose):
                print("fin non détectée d'un programme (rattrapage hors CP)")
            OnlinePTV.loc[OnlinePTV.shape[0]] = [i%1440,PTV['TITRE'].iloc[index_PTV],'non',percentage,"fin non détectée d'un programme long(rattrapage hors CP)"]
            lastend = planifiedend
            lastCP=0
            index_PTV += 1  
            index_PTV = index_PTV%(PTV.shape[0])
            lastduree = PTV['DUREE'].iloc[index_PTV]
            planifiedend = (lastend + lastduree)
        if(PTV['TITRE'].iloc[index_PTV] == 'Journal'):
            lastend = PTV['debut'].iloc[index_PTV]
        else:
            pass
            #regular point; we don't do a thing
        
        
        

    

In [15]:
OnlinePTV['Heure'] = OnlinePTV['minute'].apply(lambda x: str(int(x/60))+':'+str(x%60))
display_side_by_side(PTV,OnlinePTV)

,TITRE,DUREE,description programme,HEURE,debut,fin
0,TFou,120,Magazine jeunesse,06:25:00,385,505
1,Météo,5,Météo,08:25:00,505,510
2,Téléshopping,45,Magazine de télé-achat,08:30:00,510,555
3,Météo,5,Météo,09:15:00,555,560
4,Mon histoire vraie,10,Série réaliste,09:20:00,560,570
5,Petits secrets entre voisins,20,Série réaliste,09:30:00,570,590
6,Petits secrets entre voisins,35,Série réaliste,09:50:00,590,625
7,Demain nous appartient,35,Série dramatique,10:25:00,625,660
8,Les feux de l'amour,55,Feuilleton sentimental,11:00:00,660,715
9,Petits plats en équilibre,5,Magazine de la gastronomie,11:55:00,715,720


In [6]:
OnlinePTV.to_html('OnlinePTV.html')

In [7]:
len(label),true_labels.shape,Points.shape

(98, (98, 1), (98, 1))

In [8]:
lab = pd.DataFrame(label)

In [9]:
def mesure_class(y_pred,y_true,j):
    TP = 0
    FP = 0
    FN = 0
    for i in range(len(y_pred)):
        if(y_pred[i] == j):
            if(y_true[i] == j):
                TP += 1
            else:
                FP += 1
    for i in range(len(y_true)):
        if(y_true[i] == j):
            if(y_pred[i] == j):
                pass
            else:
                FN += 1
    return TP,FP,FN

def score(tp,fp,fn,epsilon=10**-5):
    beta = 2
    p = tp/(tp+fp+epsilon)
    r = tp/(tp+fn+epsilon)
    beta_squared = beta ** 2
    f = (beta_squared + 1) * (p * r) / (beta_squared * p + r+epsilon)
    
    print("|| precison: "+str(p)+"|| recall: "+str(r)+"|| fbeta: "+str(f))
    print('--------------------------------------------------')

def mesure(y,y_test,p1=0.25,p2=0.5):
    TP1,FP1,FN1 = mesure_class(y,y_test,0)
    TP2,FP2,FN2 = mesure_class(y,y_test,1)
    TP3,FP3,FN3 = mesure_class(y,y_test,2)
    print("pour la classe 0")
    score(TP1,FP1,FN1)
    print("pour la classe 1")
    score(TP2,FP2,FN2)
    print("pour la classe 2")
    score(TP3,FP3,FN3)

In [10]:
mesure(label,true_labels[0])

pour la classe 0
|| precison: 0.3529409688582536|| recall: 0.19354832466183075|| fbeta: 0.21276455661985622
--------------------------------------------------
pour la classe 1
|| precison: 0.5434781427221429|| recall: 0.6756754930606775|| fbeta: 0.6443274697178928
--------------------------------------------------
pour la classe 2
|| precison: 0.4571427265306495|| recall: 0.5333331555556148|| fbeta: 0.5161266805504657
--------------------------------------------------


In [11]:
len(label)

98